## Import

In [1]:
from tqdm import tnrange
from tqdm import tqdm
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook
from selenium.webdriver import Chrome
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

import time
import pandas as pd
import numpy as np
import re
import requests

# Setting

In [168]:
# 크롬 브라우저 열기
driver = Chrome(executable_path='/home/ubunt/chromedriver')

In [155]:
# 단일 환율
url = "https://kr.investing.com/currencies/single-currency-crosses"
driver.get(url)

In [156]:
download_file_path = "/home/ubunt/Downloads/"
save_file_path = "/home/ubunt/4-2/stock price/data/price/"

# Functions

In [157]:
def exchange_crawl(select_list, start_day) :
    global driver
    global download_file_path
    global save_file_path
    # 종목 이름 및 url
    tmp_names = driver.find_elements_by_css_selector("div#cross_rates_container table#cr1 tbody > tr > td.bold.left.noWrap.elp.plusIconTd a")
    urls = [x.get_attribute("href") for x in tmp_names]
    # file name
    tmp_file_name = driver.find_elements_by_css_selector("div#cross_rates_container table#cr1 tbody > tr > td.left.noWrap")
    file_name = [x.text for idx, x in enumerate(tmp_file_name) if idx % 2 == 1]
    # 과거 데이터 url 적용
    past_urls = [x + "-historical-data" for x in urls]
    
    for name, url in tqdm_notebook(zip(file_name, past_urls)) :
        try :
            if name in select_list :
                driver.get(url)
                driver.implicitly_wait(3)
                # 날짜 입력란
                tmp_date = driver.find_element_by_css_selector("div#widgetFieldDateRange")
                tmp_date.click()
                driver.implicitly_wait(3)
                tmp_intput = driver.find_element_by_css_selector("input#startDate")
                # 기존 날짜 지우기
                for _ in range(11) :
                    tmp_intput.send_keys(Keys.BACKSPACE)
                    time.sleep(0.1)
                # 2010/10/04 부터 시작
                tmp_intput.send_keys(start_day)
                # 요청 버튼
                request_btn = driver.find_element_by_css_selector("a#applyBtn")
                request_btn.click()
                time.sleep(5)

                # 다운로드 버튼
                download_btn = driver.find_element_by_css_selector("a.newBtn.LightGray.downloadBlueIcon.js-download-data")
                download_btn.click()
            else :
                continue
        except :
            pass
    ## 컬럼명 변경
    choice_file = [re.sub("/","_",x) + " 내역.csv" for x in select_list]
    for file in tqdm_notebook(choice_file) :
        a = pd.read_csv(download_file_path + file)
        a['날짜'] = [cleansing(x) for x in a['날짜']]
        new_column = ['date']
        new_column.extend([re.sub(" ","",re.split(" ",file)[0]) + "_" + del_per(x) for x in a.columns[1:]])
        a.columns = new_column
        a.to_csv(save_file_path + file, index = False)

In [170]:
def exchange_gift_crawl(start_day) :
    global driver
    global download_file_path
    global save_file_path
    global select_list
    
    whole_file_name = []
    tmp_names = driver.find_elements_by_css_selector("table#BarchartDataTable tbody tr td.bold.left.noWrap a")
    names = [x.text for x in tmp_names]
    urls = [x.get_attribute("href") for x in tmp_names]
    
    final_urls = [y for x, y in zip(names,urls) if "(P)" not in x]
    for tmp_url in tqdm_notebook(final_urls) :
        driver.get(tmp_url)
        time.sleep(3)
        tmp_urls2 = driver.find_elements_by_xpath("//ul[@id='pairSublinksLevel2']/li/a")
        past_data_idx = [idx + 1 for idx,x in enumerate(tmp_urls2) if x.text == "과거 데이터"][0]
        tmp_url = driver.find_element_by_xpath("//ul[@id='pairSublinksLevel2']/li[{}]/a".format(past_data_idx))
        assert tmp_url.text == "과거 데이터"
        tmp_url.click()
        driver.implicitly_wait(6)

        file_name = driver.find_element_by_css_selector("h2.float_lang_base_1.inlineblock").text

        if file_name in select_list :
            continue

        if "/" in file_name :
            file_name = re.sub("/","_",file_name)
        whole_file_name.append(file_name)

        # 날짜 입력란
        tmp_date = driver.find_element_by_css_selector("div#widgetFieldDateRange")
        tmp_date.click()
        driver.implicitly_wait(3)
        tmp_intput = driver.find_element_by_css_selector("input#startDate")
        # 기존 날짜 지우기
        for _ in range(11) :
            tmp_intput.send_keys(Keys.BACKSPACE)
            time.sleep(0.1)
        # 2010/10/04 부터 시작
        tmp_intput.send_keys(start_day)
        # 요청 버튼
        request_btn = driver.find_element_by_css_selector("a#applyBtn")
        request_btn.click()
        time.sleep(5)

        # 다운로드 버튼
        download_btn = driver.find_element_by_css_selector("a.newBtn.LightGray.downloadBlueIcon.js-download-data")
        download_btn.click()

    ## 컬럼명 변경
    choice_file = [x + ".csv" for x in whole_file_name]
    for file in tqdm_notebook(choice_file) :
        a = pd.read_csv(download_file_path + file)
        a['날짜'] = [cleansing(x) for x in a['날짜']]
        new_column = ['date']
        new_column.extend([re.sub(" ","",re.sub(".csv","",file)) + "_" + del_per(x) for x in a.columns[1:]])
        a.columns = new_column
        a.to_csv(save_file_path + file, index = False)

In [227]:
def bond_crawl(select_con,start_day) :
    global driver
    global download_file_path
    global save_file_path

    tmp_name_of_table = driver.find_elements_by_css_selector("table > tbody > tr > td > a")
    names = [x.text for x in tmp_name_of_table if x.text.split(" ")[0] in select_con]
    urls = [x.get_attribute("href") + "-historical-data" for x in tmp_name_of_table if x.text.split(" ")[0] in select_con]
    whole_file_name = []
    for con_url in tqdm_notebook(urls) :    
        driver.get(con_url)
        file_name = driver.find_element_by_css_selector("h2.float_lang_base_1.inlineblock").text

        if "/" in file_name :
            file_name = re.sub("/","_",file_name)
        whole_file_name.append(file_name)
        # 날짜 입력란
        tmp_date = driver.find_element_by_css_selector("div#widgetFieldDateRange")
        tmp_date.click()
        driver.implicitly_wait(3)
        tmp_intput = driver.find_element_by_css_selector("input#startDate")
        # 기존 날짜 지우기
        for _ in range(11) :
            tmp_intput.send_keys(Keys.BACKSPACE)
            time.sleep(0.1)
        # 2010/10/04 부터 시작
        tmp_intput.send_keys(start_day)

        # 요청 버튼
        request_btn = driver.find_element_by_css_selector("a#applyBtn")
        request_btn.click()
        time.sleep(5)

        # 다운로드 버튼
        download_btn = driver.find_element_by_css_selector("a.newBtn.LightGray.downloadBlueIcon.js-download-data")
        download_btn.click()

    ## 컬럼명 변경
    choice_file = [x + ".csv" for x in whole_file_name]
    for file in tqdm_notebook(choice_file) :
        a = pd.read_csv(download_file_path + file)
        a['날짜'] = [cleansing(x) for x in a['날짜']]
        new_column = ['date']
        new_column.extend([re.sub(" ","",re.sub(".csv","",file)) + "_" + del_per(x) for x in a.columns[1:]])
        a.columns = new_column
        a.to_csv(save_file_path + file, index = False)

In [158]:
def cleansing(x) :
    output = re.sub("[년월]","-",x)
    output = re.sub("[일 ]","",output)
    return(output)

In [159]:
def del_per(x) :
    try :
        output = re.sub(" %","",x)
        return(output)
    except :
        pass

## Crawling

* 환율 데이터 다운로드

In [160]:
# 전체에서 뽑고 싶은 것만
select_list = ["EUR/USD","GBP/USD","USD/JPY","USD/KRW", "USD/CNY", "USD/XPF", "USD/CAD"]

In [161]:
exchange_crawl(select_list, "2010/01/04")

/home/ubunt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


/home/ubunt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


* 통화 선물 데이터 다운로드

In [169]:
# 통화 선물
url = "https://kr.investing.com/currencies/fx-futures"
driver.get(url)

In [171]:
exchange_gift_crawl("2010/01/04")

/home/ubunt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  del sys.path[0]


/home/ubunt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:54: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


* 채권 데이터 다운로드

In [231]:
# 채권
url = "https://kr.investing.com/rates-bonds/world-government-bonds"
driver.get(url)

In [232]:
select_con = ["캐나다","미국","독일","중국","영국","일본","프랑스","한국","이탈리아"]

In [ ]:
bond_crawl(select_con, "2010/01/04")